<a href="https://colab.research.google.com/github/mohammedameen426/Bridge_Crack_Image_Data/blob/master/Predection_on_HRCDS_2025_11_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Wed Nov 12 08:47:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!unzip -q /content/drive/MyDrive/HRCDS.zip

In [8]:
!unzip -q /content/drive/MyDrive/1_Weights_on_HRCDS_dataset.zip

In [4]:
!pip3 install ezdxf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.3 MB/s eta 0:00:00


In [5]:
import tensorflow as tf
import os
import glob
import numpy as np
import shutil

import random
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm  #Progress bar for loops
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.transform import resize

import ezdxf

In [6]:
IMG_HEIGHT=256  #256
IMG_WIDTH=256  # 256
IMG_CHANNELS = 1  # Example for RGB images
GRAY = False  # Set to True for grayscale images

In [7]:
import cv2
import os

# 👇 Input directory containing your original images
input_dir = '/content/HRCDS/test_mark_color'

# 👇 Output directory where inverted images will be saved
output_dir = '/content/test_lab_inverted'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# 👇 Get list of all image files in the input directory
# (You can add more extensions like .jpg, .jpeg, .bmp if needed)
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp')
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(image_extensions)]

print(f"Found {len(image_files)} images. Processing...")

# 👇 Loop through each image, invert, and save
for image_file in image_files:
    input_path = os.path.join(input_dir, image_file)
    output_path = os.path.join(output_dir, image_file)  # same name, different folder

    # Read image in grayscale
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"⚠️ Could not read image: {image_file} — skipping.")
        continue

    # Invert the image: Black <-> White
    inverted_img = cv2.bitwise_not(img)

    # Save the inverted image
    cv2.imwrite(output_path, inverted_img)
    print(f"✅ Saved inverted image: {output_path}")

print("🎉 All images processed.")

Found 100 images. Processing...
✅ Saved inverted image: /content/test_lab_inverted/test_0078_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0029_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0063_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0093_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0089_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0077_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0067_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0017_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0084_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0075_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0079_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0008_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_0011_mask.png
✅ Saved inverted image: /content/test_lab_inverted/test_00

In [9]:
## Download predictions and trained model
!zip -r /content/test_lab_inverted.zip /content/test_lab_inverted
from google.colab import files
files.download("/content/test_lab_inverted.zip")


  adding: content/test_lab_inverted/ (stored 0%)
  adding: content/test_lab_inverted/test_0078_mask.png (deflated 31%)
  adding: content/test_lab_inverted/test_0029_mask.png (deflated 26%)
  adding: content/test_lab_inverted/test_0063_mask.png (deflated 57%)
  adding: content/test_lab_inverted/test_0093_mask.png (deflated 58%)
  adding: content/test_lab_inverted/test_0089_mask.png (deflated 28%)
  adding: content/test_lab_inverted/test_0077_mask.png (deflated 30%)
  adding: content/test_lab_inverted/test_0067_mask.png (deflated 19%)
  adding: content/test_lab_inverted/test_0017_mask.png (deflated 35%)
  adding: content/test_lab_inverted/test_0084_mask.png (deflated 27%)
  adding: content/test_lab_inverted/test_0075_mask.png (deflated 23%)
  adding: content/test_lab_inverted/test_0079_mask.png (deflated 28%)
  adding: content/test_lab_inverted/test_0008_mask.png (deflated 29%)
  adding: content/test_lab_inverted/test_0011_mask.png (deflated 50%)
  adding: content/test_lab_inverted/test_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Our model**

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------
# Architecture components
# -----------------------
# EAFFM (safe: layers created in __init__)
class EAFFM(layers.Layer):
    def __init__(self, filters, reduction_ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio

        self.conv1 = layers.Conv2D(filters, 1, padding='same', kernel_initializer='he_normal')
        self.conv3 = layers.Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')
        self.conv5 = layers.Conv2D(filters, 5, padding='same', kernel_initializer='he_normal')

        # ✅ fixed: use depthwise_initializer instead of kernel_initializer
        self.dw = layers.DepthwiseConv2D(3, padding='same', depth_multiplier=1, depthwise_initializer='he_normal')

        self.global_avg = layers.GlobalAveragePooling2D()
        self.global_max = layers.GlobalMaxPooling2D()
        self.fc1 = layers.Dense(max((filters * 4) // reduction_ratio, 4), activation='relu')
        self.fc2 = layers.Dense(filters * 4, activation='sigmoid')

        self.spatial_conv = layers.Conv2D(1, 7, padding='same', activation='sigmoid', kernel_initializer='he_normal')
        self.fusion_conv = layers.Conv2D(filters, 1, padding='same', kernel_initializer='he_normal')
        self.match_conv = layers.Conv2D(filters, 1, padding='same', kernel_initializer='he_normal')


    def call(self, x):
        inp_ch = x.shape[-1]   # ✅ static shape
        f1 = self.conv1(x)
        f3 = self.conv3(x)
        f5 = self.conv5(x)
        fd = self.dw(x)
        multi = tf.concat([f1, f3, f5, fd], axis=-1)

        # Channel attention
        gap = self.global_avg(multi)
        gmp = self.global_max(multi)
        ch = self.fc1(gap + gmp)
        ch = self.fc2(ch)
        ch = tf.reshape(ch, [-1, 1, 1, ch.shape[-1]])
        multi = multi * ch

        # Spatial attention
        sp = self.spatial_conv(multi)
        fused = self.fusion_conv(multi * sp)

        # ✅ Fixed comparison (static shape)
        if fused.shape[-1] != inp_ch:
            fused = self.match_conv(fused)

        return fused + x



# -----------------------
# PFI (progressive feature integration)
# -----------------------
class PFI(layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.conv2 = layers.Conv2D(filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.conv3 = layers.Conv2D(filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.refine = layers.Conv2D(filters, 1, padding='same', activation='relu', kernel_initializer='he_normal')
        self.attn = layers.Conv2D(filters, 1, padding='same', activation='sigmoid', kernel_initializer='he_normal')

    def call(self, x):
        inp_ch = tf.shape(x)[-1]
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        refined = self.refine(x1 + x2 + x3)
        att = self.attn(refined)
        if refined.shape[-1] != inp_ch:
            refined = layers.Conv2D(inp_ch, 1, padding='same')(refined)
        return x * att + refined


# -----------------------
# MSCAT (multi-scale local conv + MHA transformer)
# -----------------------
class MSCAT(layers.Layer):
    def __init__(self, dim, num_heads=4, ff_dim=192, expansion_ratio=2, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.num_heads = num_heads
        self.expansion_ratio = expansion_ratio

        self.dw1 = layers.Conv2D(dim, 3, padding='same', dilation_rate=1, kernel_initializer='he_normal')
        self.dw2 = layers.Conv2D(dim, 3, padding='same', dilation_rate=2, kernel_initializer='he_normal')
        self.dw3 = layers.Conv2D(dim, 3, padding='same', dilation_rate=4, kernel_initializer='he_normal')
        self.fuse = layers.Conv2D(dim, 1, activation='relu', kernel_initializer='he_normal')

        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=max(dim // num_heads, 1), dropout=0.1)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

        self.ffn_gate_dense = layers.Dense(ff_dim * expansion_ratio, activation='gelu')
        self.ffn_gate_proj = layers.Dense(dim, activation='sigmoid')
        self.ffn_val_dense = layers.Dense(ff_dim * expansion_ratio, activation='gelu')
        self.ffn_val_proj = layers.Dense(dim)
        self.dropout = layers.Dropout(0.1)

    def call(self, x, training=False):
        B = tf.shape(x)[0]
        H = tf.shape(x)[1]
        W = tf.shape(x)[2]
        C = tf.shape(x)[3]

        s1 = self.dw1(x)
        s2 = self.dw2(x)
        s3 = self.dw3(x)
        multi = self.fuse(s1 + s2 + s3)

        x_flat = tf.reshape(multi, [B, H * W, C])
        attn_out = self.mha(x_flat, x_flat, training=training)
        attn_out = self.dropout(attn_out, training=training)
        attn_out = tf.reshape(attn_out, [B, H, W, C])
        x_comb = self.norm1(multi + attn_out)

        x_flat2 = tf.reshape(x_comb, [B, H * W, C])
        gate = self.ffn_gate_proj(self.ffn_gate_dense(x_flat2))
        val = self.ffn_val_proj(self.ffn_val_dense(x_flat2))
        gated = gate * val
        gated = tf.reshape(gated, [B, H, W, C])
        out = self.norm2(x_comb + gated)
        return out


# -----------------------
# Model builder (UNet-like with deep supervision)
# -----------------------
def build_model(input_shape=(256,256,1), base_filters=8):
    inputs = layers.Input(shape=input_shape, name="input_image")
    x = layers.GaussianNoise(0.08)(inputs)

    # Encoder
    c1 = layers.Conv2D(base_filters, 3, padding='same', activation='relu')(x)
    c1 = layers.Conv2D(base_filters, 3, padding='same', activation='relu')(c1)
    p1 = layers.MaxPooling2D()(c1)
    s1 = EAFFM(base_filters)(c1)

    c2 = layers.Conv2D(base_filters*2, 3, padding='same', activation='relu')(p1)
    c2 = layers.Conv2D(base_filters*2, 3, padding='same', activation='relu')(c2)
    p2 = layers.MaxPooling2D()(c2)
    s2 = EAFFM(base_filters*2)(c2)

    c3 = layers.Conv2D(base_filters*4, 3, padding='same', activation='relu')(p2)
    c3 = layers.Conv2D(base_filters*4, 3, padding='same', activation='relu')(c3)
    p3 = layers.MaxPooling2D()(c3)
    s3 = EAFFM(base_filters*4)(c3)

    # Bottleneck
    b = layers.Conv2D(base_filters*8, 3, padding='same', activation='relu')(p3)
    b = layers.Conv2D(base_filters*8, 3, padding='same', activation='relu')(b)
    b = MSCAT(base_filters*8, num_heads=4, ff_dim=128)(b)

    # Decoder
    u3 = layers.Conv2DTranspose(base_filters*4, 2, strides=2, padding='same', activation='relu')(b)
    u3 = layers.Concatenate()([u3, s3])
    u3 = layers.Conv2D(base_filters*4, 3, padding='same', activation='relu')(u3)
    u3 = layers.Conv2D(base_filters*4, 3, padding='same', activation='relu')(u3)

    u2 = layers.Conv2DTranspose(base_filters*2, 2, strides=2, padding='same', activation='relu')(u3)
    u2 = layers.Concatenate()([u2, s2])
    u2 = layers.Conv2D(base_filters*2, 3, padding='same', activation='relu')(u2)
    u2 = layers.Conv2D(base_filters*2, 3, padding='same', activation='relu')(u2)

    u1 = layers.Conv2DTranspose(base_filters, 2, strides=2, padding='same', activation='relu')(u2)
    u1 = layers.Concatenate()([u1, s1])
    u1 = layers.Conv2D(base_filters, 3, padding='same', activation='relu')(u1)
    u1 = layers.Conv2D(base_filters, 3, padding='same', activation='relu')(u1)

    # Final outputs
    final_conv = layers.Conv2D(1, 1, padding='same')(u1)
    final_sig = layers.Activation('sigmoid', name='mask')(final_conv)

    ds2 = layers.Conv2D(1, 1, padding='same')(u2)
    ds2_up = layers.UpSampling2D(size=(2,2), interpolation='bilinear')(ds2)
    ds2_sig = layers.Activation('sigmoid', name='ds2')(ds2_up)

    ds3 = layers.Conv2D(1, 1, padding='same')(u3)
    ds3_up = layers.UpSampling2D(size=(4,4), interpolation='bilinear')(ds3)
    ds3_sig = layers.Activation('sigmoid', name='ds3')(ds3_up)

    model = models.Model(inputs=inputs, outputs=[final_sig, ds2_sig, ds3_sig], name="crack_unet_deepsup")
    return model


# Re-build the model with the correct input shape after IMG_CHANNELS has been updated
model = build_model(input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm_2', however the layer does not have a `build(

Model: "crack_unet_deepsup"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_noise      │ (None, 256, 256,  │          0 │ input_image[0][0] │
│ (GaussianNoise)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │         80 │ gaussian_noise[0… │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │        584 │ conv2d[0][0]      │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │      1,168 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │      2,320 │ conv2d_8[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 64, 64,    │      4,640 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 64, 64,    │      9,248 │ conv2d_16[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_17[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 32, 32,    │     18,496 │ max_pooling2d_2[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 32, 32,    │     36,928 │ conv2d_24[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mscat (MSCAT)       │ (None, 32, 32,    │    198,016 │ conv2d_25[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 64, 64,    │      8,224 │ mscat[0][0]       │
│ (Conv2DTranspose)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eaffm_2 (EAFFM)     │ (None, 64, 64,    │     50,897 │ conv2d_17[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 64,    │          0 │ conv2d_transpose

 Total params: 388,554 (1.48 MB)

 Trainable params: 388,554 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
IMG_HEIGHT=256  #256
IMG_WIDTH=256  # 256
IMG_CHANNELS = 1  # Example for grayscale images
GRAY = True  # Set to True for grayscale images

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1  # ✅ match grayscale training

# (include your EAFFM, PFI, MSCAT, build_model definitions here)

# Build and load
model = build_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
model.load_weights('/content/1_Weights on DeepCrack dataset/Our_model_on_HRCDS_2025.11.7.h5')
print("✅ Weights loaded successfully.")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'eaffm_5', however the layer does not have a `buil

✅ Weights loaded successfully.


In [13]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [14]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [15]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re

# ================================
# Parameters - using global variables
# ================================
# No need to redefine GRAY, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS here as they are global.

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

# Dynamically get and sort files from the directory
test_files_list = sorted(os.listdir(folder_path), key=extract_number)
test_files3 = [os.path.join(folder_path, f) for f in test_files_list]

X_test = []
processed_filenames = [] # Keep track of filenames that were successfully processed

print(f"Found {len(test_files3)} images to process.")

for file_path in test_files3:
    img = None
    if GRAY:
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # Specify grayscale load
    else:
        img = cv2.imread(file_path, cv2.IMREAD_COLOR) # Specify color load

    if img is None:
        print(f"⚠️ Could not read image: {os.path.basename(file_path)} — skipping.")
        continue # Skip to the next file if image could not be read

    # Resize and reshape
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    if GRAY:
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    # For RGB, ensure it's (H, W, C) if it became 2D after resize
    elif img.ndim == 2 and IMG_CHANNELS == 3: # If resized color image is 2D
        img = np.stack([img]*IMG_CHANNELS, axis=-1)

    X_test.append(img)
    processed_filenames.append(os.path.basename(file_path))


if not X_test:
    print("❌ No images were successfully loaded for prediction. Exiting.")
    raise ValueError("No images loaded for prediction.")

X_test = np.array(X_test)
print(f"✅ Successfully loaded and preprocessed {len(X_test)} images.")

# ================================
# Make predictions using your model
# ================================
# The model outputs a list of predictions for each head. We're interested in the first one (main mask).
all_predictions = model.predict(X_test)
main_mask_predictions = all_predictions[0] # Get predictions from the first output head


# Save results (no display)
# ================================
output_dir = "/content/predictions_our_model"
os.makedirs(output_dir, exist_ok=True)

# Now, iterate over each individual image's prediction
print(f"Saving {len(main_mask_predictions)} predictions to {output_dir}...")
for i in range(len(main_mask_predictions)):
    prediction = main_mask_predictions[i] # Get the prediction for the current image
    pred_display = prediction.squeeze()   # Shape becomes (IMG_HEIGHT, IMG_WIDTH)
    inverted_prediction = 1.0 - pred_display

    # Use original filename for better traceability or a clear index
    original_filename = processed_filenames[i] # Use the filename from the successfully processed list
    # Generate a consistent output filename
    output_filename = f"prediction_{os.path.splitext(original_filename)[0]}.png"
    output_path = os.path.join(output_dir, output_filename)

    # Save the 2D grayscale image
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("🎉 All predictions saved (black crack on white background). No images displayed.")

Found 100 images to process.
✅ Successfully loaded and preprocessed 100 images.
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step
Saving 100 predictions to /content/predictions_our_model...
🎉 All predictions saved (black crack on white background). No images displayed.


In [16]:
## Download predictions and trained model
!zip -r /content/predictions_our_model.zip /content/predictions_our_model
from google.colab import files
files.download("/content/predictions_our_model.zip")


  adding: content/predictions_our_model/ (stored 0%)
  adding: content/predictions_our_model/prediction_test_0022.png (deflated 4%)
  adding: content/predictions_our_model/prediction_test_0064.png (deflated 4%)
  adding: content/predictions_our_model/prediction_test_0025.png (deflated 4%)
  adding: content/predictions_our_model/prediction_test_0015.png (deflated 2%)
  adding: content/predictions_our_model/prediction_test_0063.png (deflated 1%)
  adding: content/predictions_our_model/prediction_test_0007.png (deflated 2%)
  adding: content/predictions_our_model/prediction_test_0026.png (deflated 7%)
  adding: content/predictions_our_model/prediction_test_0094.png (deflated 3%)
  adding: content/predictions_our_model/prediction_test_0021.png (deflated 2%)
  adding: content/predictions_our_model/prediction_test_0018.png (deflated 3%)
  adding: content/predictions_our_model/prediction_test_0014.png (deflated 3%)
  adding: content/predictions_our_model/prediction_test_0042.png (deflated 4%)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Unet**

In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def UNetSmall(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    e1 = _conv_block_unet(inputs, 16)  # Reduced from 32 to 16
    e2 = _conv_block_unet(layers.MaxPooling2D(2)(e1), 32)  # Reduced from 64 to 32
    e3 = _conv_block_unet(layers.MaxPooling2D(2)(e2), 64)  # Reduced from 128 to 64

    # Bridge
    bridge = _conv_block_unet(layers.MaxPooling2D(2)(e3), 128)  # Reduced from 256 to 128

    # Decoder
    d3 = layers.Conv2DTranspose(64, 2, strides=2)(bridge)  # Reduced from 128 to 64
    d3 = layers.Concatenate()([d3, e3])
    d3 = _conv_block_unet(d3, 64)  # Reduced from 128 to 64

    d2 = layers.Conv2DTranspose(32, 2, strides=2)(d3)  # Reduced from 64 to 32
    d2 = layers.Concatenate()([d2, e2])
    d2 = _conv_block_unet(d2, 32)  # Reduced from 64 to 32

    d1 = layers.Conv2DTranspose(16, 2, strides=2)(d2)  # Reduced from 32 to 16
    d1 = layers.Concatenate()([d1, e1])
    d1 = _conv_block_unet(d1, 16)  # Reduced from 32 to 16

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    model.summary()

    return model

def _conv_block_unet(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

# The model
UNET = UNetSmall()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 256, 256,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │         64 │ conv2d_78[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 256, 256,  │      2,320 │ re_lu[0][0]       │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_79[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 128, 128,  │          0 │ re_lu_1[0][0]     │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 128, 128,  │      4,640 │ max_pooling2d_6[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_80[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 128, 128,  │      9,248 │ re_lu_2[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_81[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 64, 64,    │          0 │ re_lu_3[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_82 (Conv2D)  │ (None, 64, 64,    │     18,496 │ max_pooling2d_7[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_82[0][0] 

 Total params: 484,561 (1.85 MB)

 Trainable params: 483,153 (1.84 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
UNET.load_weights('/content/1_Weights on DeepCrack dataset/UNet_UNetSmall_on_HRCDS_2025.11.10.h5')


In [21]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/Deepcrack_2024.8.31/test/test_img/1.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/2.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/3.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/4.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/5.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/6.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/7.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/8.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/9.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/10.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/11.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/12.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/13.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/14.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/15.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/16.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/17.jpg',
 '/content/Deepcrack_2024.8.31/test/test_img/18.jpg',
 '/content/Deepcrack_2024.8.31/test/t

In [22]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [23]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = UNET.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_UNet"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 908ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [24]:
## Download predictions and trained model
!zip -r /content/predection_of_UNet.zip /content/predection_of_UNet
from google.colab import files
files.download("/content/predection_of_UNet.zip")


  adding: content/predection_of_UNet/ (stored 0%)
  adding: content/predection_of_UNet/image_3.png (deflated 4%)
  adding: content/predection_of_UNet/image_63.png (deflated 4%)
  adding: content/predection_of_UNet/image_84.png (deflated 5%)
  adding: content/predection_of_UNet/image_16.png (deflated 3%)
  adding: content/predection_of_UNet/image_2.png (deflated 3%)
  adding: content/predection_of_UNet/image_12.png (deflated 6%)
  adding: content/predection_of_UNet/image_31.png (deflated 4%)
  adding: content/predection_of_UNet/image_9.png (deflated 8%)
  adding: content/predection_of_UNet/image_21.png (deflated 4%)
  adding: content/predection_of_UNet/image_69.png (deflated 6%)
  adding: content/predection_of_UNet/image_77.png (deflated 4%)
  adding: content/predection_of_UNet/image_75.png (deflated 4%)
  adding: content/predection_of_UNet/image_51.png (deflated 4%)
  adding: content/predection_of_UNet/image_48.png (deflated 6%)
  adding: content/predection_of_UNet/image_36.png (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Attention UNet**

In [27]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def UNetSmall(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    e1 = _conv_block_unet(inputs, 16)  # Reduced from 32 to 16
    e2 = _conv_block_unet(layers.MaxPooling2D(2)(e1), 32)  # Reduced from 64 to 32
    e3 = _conv_block_unet(layers.MaxPooling2D(2)(e2), 64)  # Reduced from 128 to 64

    # Bridge
    bridge = _conv_block_unet(layers.MaxPooling2D(2)(e3), 128)  # Reduced from 256 to 128

    # Decoder
    d3 = layers.Conv2DTranspose(64, 2, strides=2)(bridge)  # Reduced from 128 to 64
    d3 = layers.Concatenate()([d3, e3])
    d3 = _conv_block_unet(d3, 64)  # Reduced from 128 to 64

    d2 = layers.Conv2DTranspose(32, 2, strides=2)(d3)  # Reduced from 64 to 32
    d2 = layers.Concatenate()([d2, e2])
    d2 = _conv_block_unet(d2, 32)  # Reduced from 64 to 32

    d1 = layers.Conv2DTranspose(16, 2, strides=2)(d2)  # Reduced from 32 to 16
    d1 = layers.Concatenate()([d1, e1])
    d1 = _conv_block_unet(d1, 16)  # Reduced from 32 to 16

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    model.summary()

    return model

def _conv_block_unet(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

#The model
attention_unet=UNetSmall()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_124 (Conv2D) │ (None, 256, 256,  │        160 │ input_layer_2[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_124[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_14 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_125 (Conv2D) │ (None, 256, 256,  │      2,320 │ re_lu_14[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_125[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 128, 128,  │          0 │ re_lu_15[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_126 (Conv2D) │ (None, 128, 128,  │      4,640 │ max_pooling2d_13… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_126[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_16 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_127 (Conv2D) │ (None, 128, 128,  │      9,248 │ re_lu_16[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_127[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_17 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 64, 64,    │          0 │ re_lu_17[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_128 (Conv2D) │ (None, 64, 64,    │     18,496 │ max_pooling2d_14… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_128[0][0]

 Total params: 484,561 (1.85 MB)

 Trainable params: 483,153 (1.84 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [28]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
attention_unet.load_weights('/content/1_Weights on DeepCrack dataset/attention_unet_UNetSmall_on_HRCDS_2025.11.10.h5')


In [29]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [30]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [31]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = attention_unet.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_attentionUNet"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 414ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [32]:
## Download predictions and trained model
!zip -r /content/predection_of_attentionUNet.zip /content/predection_of_attentionUNet
from google.colab import files
files.download("/content/predection_of_attentionUNet.zip")


  adding: content/predection_of_attentionUNet/ (stored 0%)
  adding: content/predection_of_attentionUNet/image_3.png (deflated 8%)
  adding: content/predection_of_attentionUNet/image_63.png (deflated 13%)
  adding: content/predection_of_attentionUNet/image_84.png (deflated 6%)
  adding: content/predection_of_attentionUNet/image_16.png (deflated 7%)
  adding: content/predection_of_attentionUNet/image_2.png (deflated 6%)
  adding: content/predection_of_attentionUNet/image_12.png (deflated 6%)
  adding: content/predection_of_attentionUNet/image_31.png (deflated 5%)
  adding: content/predection_of_attentionUNet/image_9.png (deflated 16%)
  adding: content/predection_of_attentionUNet/image_21.png (deflated 5%)
  adding: content/predection_of_attentionUNet/image_69.png (deflated 6%)
  adding: content/predection_of_attentionUNet/image_77.png (deflated 4%)
  adding: content/predection_of_attentionUNet/image_75.png (deflated 5%)
  adding: content/predection_of_attentionUNet/image_51.png (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ResBCU_net**

In [33]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def UNetSmall(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    e1 = _conv_block_unet(inputs, 16)  # Reduced from 32 to 16
    e2 = _conv_block_unet(layers.MaxPooling2D(2)(e1), 32)  # Reduced from 64 to 32
    e3 = _conv_block_unet(layers.MaxPooling2D(2)(e2), 64)  # Reduced from 128 to 64

    # Bridge
    bridge = _conv_block_unet(layers.MaxPooling2D(2)(e3), 128)  # Reduced from 256 to 128

    # Decoder
    d3 = layers.Conv2DTranspose(64, 2, strides=2)(bridge)  # Reduced from 128 to 64
    d3 = layers.Concatenate()([d3, e3])
    d3 = _conv_block_unet(d3, 64)  # Reduced from 128 to 64

    d2 = layers.Conv2DTranspose(32, 2, strides=2)(d3)  # Reduced from 64 to 32
    d2 = layers.Concatenate()([d2, e2])
    d2 = _conv_block_unet(d2, 32)  # Reduced from 64 to 32

    d1 = layers.Conv2DTranspose(16, 2, strides=2)(d2)  # Reduced from 32 to 16
    d1 = layers.Concatenate()([d1, e1])
    d1 = _conv_block_unet(d1, 16)  # Reduced from 32 to 16

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    model.summary()

    return model

def _conv_block_unet(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

#The model
ResBCU_net = UNetSmall()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_139 (Conv2D) │ (None, 256, 256,  │        160 │ input_layer_3[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_139[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_28 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_140 (Conv2D) │ (None, 256, 256,  │      2,320 │ re_lu_28[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_140[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_29 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 128, 128,  │          0 │ re_lu_29[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_141 (Conv2D) │ (None, 128, 128,  │      4,640 │ max_pooling2d_16… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_141[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_30 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_142 (Conv2D) │ (None, 128, 128,  │      9,248 │ re_lu_30[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_142[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_31 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 64, 64,    │          0 │ re_lu_31[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_143 (Conv2D) │ (None, 64, 64,    │     18,496 │ max_pooling2d_17… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_143[0][0]

 Total params: 484,561 (1.85 MB)

 Trainable params: 483,153 (1.84 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
ResBCU_net.load_weights('/content/1_Weights on DeepCrack dataset/ResBCU-Net_UNetSmall_on_HRCDS_2025.11.10.h5')


In [35]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [36]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [37]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = ResBCU_net.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_ResBCU_net"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 601ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [38]:
## Download predictions and trained model
!zip -r /content/predection_of_ResBCU_net.zip /content/predection_of_ResBCU_net
from google.colab import files
files.download("/content/predection_of_ResBCU_net.zip")


  adding: content/predection_of_ResBCU_net/ (stored 0%)
  adding: content/predection_of_ResBCU_net/image_3.png (deflated 5%)
  adding: content/predection_of_ResBCU_net/image_63.png (deflated 4%)
  adding: content/predection_of_ResBCU_net/image_84.png (deflated 5%)
  adding: content/predection_of_ResBCU_net/image_16.png (deflated 5%)
  adding: content/predection_of_ResBCU_net/image_2.png (deflated 4%)
  adding: content/predection_of_ResBCU_net/image_12.png (deflated 7%)
  adding: content/predection_of_ResBCU_net/image_31.png (deflated 5%)
  adding: content/predection_of_ResBCU_net/image_9.png (deflated 11%)
  adding: content/predection_of_ResBCU_net/image_21.png (deflated 5%)
  adding: content/predection_of_ResBCU_net/image_69.png (deflated 6%)
  adding: content/predection_of_ResBCU_net/image_77.png (deflated 3%)
  adding: content/predection_of_ResBCU_net/image_75.png (deflated 4%)
  adding: content/predection_of_ResBCU_net/image_51.png (deflated 4%)
  adding: content/predection_of_ResB

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **SegNet**

In [39]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_segnet(imagesize=(IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS)):

    inputs = layers.Input(imagesize) #output  # Use layers.Input

    # Encoder (Downsampling) with Conv + BatchNormalization + ReLU + MaxPooling
    # Block 1
    x1 = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)
    x1_pool = layers.MaxPooling2D((2, 2))(x1)

    # Block 2
    x2 = layers.Conv2D(128, (3, 3), padding='same')(x1_pool)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.ReLU()(x2)
    x2_pool = layers.MaxPooling2D((2, 2))(x2)

    # Block 3
    x3 = layers.Conv2D(256, (3, 3), padding='same')(x2_pool)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.ReLU()(x3)
    x3_pool = layers.MaxPooling2D((2, 2))(x3)

    # Block 4
    x4 = layers.Conv2D(512, (3, 3), padding='same')(x3_pool)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.ReLU()(x4)
    x4_pool = layers.MaxPooling2D((2, 2))(x4)

    # Decoder (Upsampling) with Conv layers
    # Block 1
    x5 = layers.UpSampling2D(size=(2, 2))(x4_pool)
    x5 = layers.Conv2D(512, (3, 3), padding='same')(x5)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.ReLU()(x5)

    # Block 2
    x6 = layers.UpSampling2D(size=(2, 2))(x5)
    x6 = layers.Conv2D(256, (3, 3), padding='same')(x6)
    x6 = layers.BatchNormalization()(x6)
    x6 = layers.ReLU()(x6)

    # Block 3
    x7 = layers.UpSampling2D(size=(2, 2))(x6)
    x7 = layers.Conv2D(128, (3, 3), padding='same')(x7)
    x7 = layers.BatchNormalization()(x7)
    x7 = layers.ReLU()(x7)

    # Block 4
    x8 = layers.UpSampling2D(size=(2, 2))(x7)
    x8 = layers.Conv2D(64, (3, 3), padding='same')(x8)
    x8 = layers.BatchNormalization()(x8)
    x8 = layers.ReLU()(x8)
    # Output layer (segmentation)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x8)  # Change to 'softmax' for multi-class segmentation

    # Define the model
    model = models.Model(inputs=inputs, outputs=outputs)  # Use models.Model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"]) # or metrics=metrics if those are defined elsewhere

    model.summary()
    return model

model_SegNet = build_segnet()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_154 (Conv2D)             │ (None, 256, 256, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_42 (ReLU)                 │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_155 (Conv2D)             │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_43 (ReLU)                 │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_156 (Conv2D)             │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_44 (ReLU)                 │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_157 (Conv2D)             │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 32, 32, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_45 (ReLU)                 │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_158 (Conv2D)             │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 32, 32, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_46 (ReLU)                 │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 64, 64, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,466,113 (20.85 MB)

 Trainable params: 5,462,273 (20.84 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [41]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
model_SegNet.load_weights('/content/1_Weights on DeepCrack dataset/SegNet_on_HRCDS_2025.11.5.h5')


In [42]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [43]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [44]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = model_SegNet.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_model_SegNet"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step
✅ All predictions saved (black crack on white background). No images displayed.


In [45]:
## Download predictions and trained model
!zip -r /content/predection_of_model_SegNet.zip /content/predection_of_model_SegNet
from google.colab import files
files.download("/content/predection_of_model_SegNet.zip")


  adding: content/predection_of_model_SegNet/ (stored 0%)
  adding: content/predection_of_model_SegNet/image_3.png (deflated 4%)
  adding: content/predection_of_model_SegNet/image_63.png (deflated 4%)
  adding: content/predection_of_model_SegNet/image_84.png (deflated 6%)
  adding: content/predection_of_model_SegNet/image_16.png (deflated 6%)
  adding: content/predection_of_model_SegNet/image_2.png (deflated 3%)
  adding: content/predection_of_model_SegNet/image_12.png (deflated 6%)
  adding: content/predection_of_model_SegNet/image_31.png (deflated 5%)
  adding: content/predection_of_model_SegNet/image_9.png (deflated 9%)
  adding: content/predection_of_model_SegNet/image_21.png (deflated 4%)
  adding: content/predection_of_model_SegNet/image_69.png (deflated 6%)
  adding: content/predection_of_model_SegNet/image_77.png (deflated 5%)
  adding: content/predection_of_model_SegNet/image_75.png (deflated 5%)
  adding: content/predection_of_model_SegNet/image_51.png (deflated 4%)
  adding:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **FCN**

In [46]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [47]:
from tensorflow.keras import layers, models  # Import layers and models


def build_fcn_with_skip_connections(imagesize=(IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS)):

    inputs = layers.Input(imagesize) #output  # Use layers.Input
    # Encoder
    # Encoder (Downsampling) - Convolution + Max Pooling
    x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x1_pool = layers.MaxPooling2D((2, 2))(x1)

    x2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x1_pool)
    x2_pool = layers.MaxPooling2D((2, 2))(x2)

    x3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x2_pool)
    x3_pool = layers.MaxPooling2D((2, 2))(x3)

    x4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x3_pool)
    x4_pool = layers.MaxPooling2D((2, 2))(x4)

    # Decoder (Upsampling) - Using Skip Connections
    x5 = layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(x4_pool)
    x5 = layers.concatenate([x5, x4])  # Skip connection

    x6 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(x5)
    x6 = layers.concatenate([x6, x3])  # Skip connection

    x7 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x6)
    x7 = layers.concatenate([x7, x2])  # Skip connection

    x8 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x7)
    x8 = layers.concatenate([x8, x1])  # Skip connection

    # Output layer (segmentation)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x8)  # Change to 'softmax' for multi-class segmentation

    # Define the model
    model = models.Model(inputs=inputs, outputs=outputs)  # Use models.Model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"]) # or metrics=metrics if those are defined elsewhere

    model.summary()
    return model

#The model
model_FCN=build_fcn_with_skip_connections() # Pass input_shape to the function call

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_163 (Conv2D) │ (None, 256, 256,  │        640 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_23    │ (None, 128, 128,  │          0 │ conv2d_163[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_164 (Conv2D) │ (None, 128, 128,  │     73,856 │ max_pooling2d_23… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_24    │ (None, 64, 64,    │          0 │ conv2d_164[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_165 (Conv2D) │ (None, 64, 64,    │    295,168 │ max_pooling2d_24… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_25    │ (None, 32, 32,    │          0 │ conv2d_165[0][0]  │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_166 (Conv2D) │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_25… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_26    │ (None, 16, 16,    │          0 │ conv2d_166[0][0]  │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_19 │ (None, 32, 32,    │  2,359,808 │ max_pooling2d_26… │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_19      │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 1024)             │            │ conv2d_166[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_20 │ (None, 64, 64,    │  2,359,552 │ concatenate_19[0… │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, 64, 64,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_165[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_21 │ (None, 128, 128,  │    589,952 │ concatenate_20[0… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_164[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_22 │ (None, 256, 256,  │    147,520 │ concatenate_21[0… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_22      │ (None, 256, 256,  │          0 │ conv2d_transpose

 Total params: 7,006,785 (26.73 MB)

 Trainable params: 7,006,785 (26.73 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
model_FCN.load_weights('/content/1_Weights on DeepCrack dataset/FCN_on_HRCDS_2025.11.5.h5')


In [49]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [50]:
X_test5= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test5= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [51]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = model_FCN.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_model_FCN"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step
✅ All predictions saved (black crack on white background). No images displayed.


In [56]:
## Download predictions and trained model
!zip -r /content/predection_of_model_FCN.zip /content/predection_of_model_FCN
from google.colab import files
files.download("/content/predection_of_model_FCN.zip")


  adding: content/predection_of_model_FCN/ (stored 0%)
  adding: content/predection_of_model_FCN/image_3.png (deflated 4%)
  adding: content/predection_of_model_FCN/image_63.png (deflated 4%)
  adding: content/predection_of_model_FCN/image_84.png (deflated 6%)
  adding: content/predection_of_model_FCN/image_16.png (deflated 5%)
  adding: content/predection_of_model_FCN/image_2.png (deflated 4%)
  adding: content/predection_of_model_FCN/image_12.png (deflated 6%)
  adding: content/predection_of_model_FCN/image_31.png (deflated 6%)
  adding: content/predection_of_model_FCN/image_9.png (deflated 7%)
  adding: content/predection_of_model_FCN/image_21.png (deflated 5%)
  adding: content/predection_of_model_FCN/image_69.png (deflated 5%)
  adding: content/predection_of_model_FCN/image_77.png (deflated 4%)
  adding: content/predection_of_model_FCN/image_75.png (deflated 6%)
  adding: content/predection_of_model_FCN/image_51.png (deflated 3%)
  adding: content/predection_of_model_FCN/image_48.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ENet**

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [57]:
from tensorflow.keras import layers, models  # Import layers and models


def build_fcn_with_skip_connections(imagesize=(IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS)):

    inputs = layers.Input(imagesize) #output  # Use layers.Input
    # Encoder
    # Encoder (Downsampling) - Convolution + Max Pooling
    x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x1_pool = layers.MaxPooling2D((2, 2))(x1)

    x2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x1_pool)
    x2_pool = layers.MaxPooling2D((2, 2))(x2)

    x3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x2_pool)
    x3_pool = layers.MaxPooling2D((2, 2))(x3)

    x4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x3_pool)
    x4_pool = layers.MaxPooling2D((2, 2))(x4)

    # Decoder (Upsampling) - Using Skip Connections
    x5 = layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(x4_pool)
    x5 = layers.concatenate([x5, x4])  # Skip connection

    x6 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(x5)
    x6 = layers.concatenate([x6, x3])  # Skip connection

    x7 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x6)
    x7 = layers.concatenate([x7, x2])  # Skip connection

    x8 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x7)
    x8 = layers.concatenate([x8, x1])  # Skip connection

    # Output layer (segmentation)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x8)  # Change to 'softmax' for multi-class segmentation

    # Define the model
    model = models.Model(inputs=inputs, outputs=outputs)  # Use models.Model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"]) # or metrics=metrics if those are defined elsewhere

    model.summary()
    return model

#The model
ENet=build_fcn_with_skip_connections() # Pass input_shape to the function call


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_229 (Conv2D) │ (None, 256, 256,  │        640 │ input_layer_7[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_28    │ (None, 128, 128,  │          0 │ conv2d_229[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_230 (Conv2D) │ (None, 128, 128,  │     73,856 │ max_pooling2d_28… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_29    │ (None, 64, 64,    │          0 │ conv2d_230[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_231 (Conv2D) │ (None, 64, 64,    │    295,168 │ max_pooling2d_29… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_30    │ (None, 32, 32,    │          0 │ conv2d_231[0][0]  │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_232 (Conv2D) │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_30… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_31    │ (None, 16, 16,    │          0 │ conv2d_232[0][0]  │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_26 │ (None, 32, 32,    │  2,359,808 │ max_pooling2d_31… │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_24      │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 1024)             │            │ conv2d_232[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_27 │ (None, 64, 64,    │  2,359,552 │ concatenate_24[0… │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_25      │ (None, 64, 64,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_231[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_28 │ (None, 128, 128,  │    589,952 │ concatenate_25[0… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_26      │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_230[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_29 │ (None, 256, 256,  │    147,520 │ concatenate_26[0… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 256, 256,  │          0 │ conv2d_transpose

 Total params: 7,006,785 (26.73 MB)

 Trainable params: 7,006,785 (26.73 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
ENet.load_weights('/content/1_Weights on DeepCrack dataset/ENet_FCN_on_HRCDS_2025.11.10.h5')


In [59]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [60]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [61]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = ENet.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_ENet"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 326ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [62]:
## Download predictions and trained model
!zip -r /content/predection_of_ENet.zip /content/predection_of_ENet
from google.colab import files
files.download("/content/predection_of_ENet.zip")


  adding: content/predection_of_ENet/ (stored 0%)
  adding: content/predection_of_ENet/image_3.png (deflated 5%)
  adding: content/predection_of_ENet/image_63.png (deflated 4%)
  adding: content/predection_of_ENet/image_84.png (deflated 6%)
  adding: content/predection_of_ENet/image_16.png (deflated 4%)
  adding: content/predection_of_ENet/image_2.png (deflated 3%)
  adding: content/predection_of_ENet/image_12.png (deflated 6%)
  adding: content/predection_of_ENet/image_31.png (deflated 6%)
  adding: content/predection_of_ENet/image_9.png (deflated 6%)
  adding: content/predection_of_ENet/image_21.png (deflated 5%)
  adding: content/predection_of_ENet/image_69.png (deflated 4%)
  adding: content/predection_of_ENet/image_77.png (deflated 3%)
  adding: content/predection_of_ENet/image_75.png (deflated 6%)
  adding: content/predection_of_ENet/image_51.png (deflated 4%)
  adding: content/predection_of_ENet/image_48.png (deflated 5%)
  adding: content/predection_of_ENet/image_36.png (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **CFPBlock**

In [63]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [64]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def CFPBlock(x, reduction_ratio=2):
    """Context Feature Pyramid Block"""
    input_channels = x.shape[-1]
    reduced_channels = max(8, input_channels // reduction_ratio)

    # Main convolution path
    main = layers.Conv2D(input_channels, 3, padding='same', use_bias=False)(x)
    main = layers.BatchNormalization()(main)

    # Context path with channel attention
    context = layers.GlobalAveragePooling2D(keepdims=True)(x)
    context = layers.Conv2D(reduced_channels, 1, use_bias=False)(context)
    context = layers.BatchNormalization()(context)
    context = layers.ReLU()(context)
    context = layers.Conv2D(input_channels, 1, use_bias=False)(context)
    context = layers.BatchNormalization()(context)
    context = layers.Activation('sigmoid')(context)

    # Apply attention and residual connection
    attended = layers.Multiply()([main, context])
    out = layers.Add()([main, attended])
    return layers.ReLU()(out)

def build_model_1channel(input_shape=(256, 256, 1), num_classes=1):
    """Build model for 1-channel input targeting ~0.547M parameters"""
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs

    # Layer 1: 3×3 Conv, stride 2, 24 channels
    x = layers.Conv2D(24, 3, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Layer 2: 3×3 Conv, stride 1, 24 channels
    x = layers.Conv2D(24, 3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Layer 3: 3×3 Conv, stride 1, 24 channels
    x = layers.Conv2D(24, 3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Layer 4: Downsampling to 48 channels
    x = layers.Conv2D(48, 3, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Layers 5-6: 2×CFP with rK=2, 48 channels
    for _ in range(2):
        x = CFPBlock(x, reduction_ratio=2)

    # Layer 7: Downsampling to 96 channels
    x = layers.Conv2D(96, 3, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Layers 8-9: 2×CFP with rK=4, 96 channels
    for _ in range(2):
        x = CFPBlock(x, reduction_ratio=4)

    # Layers 10-11: 2×CFP with rK=8, 96 channels
    for _ in range(2):
        x = CFPBlock(x, reduction_ratio=8)

    # Layers 12-13: 2×CFP with rK=16, 96 channels
    for _ in range(2):
        x = CFPBlock(x, reduction_ratio=16)

    # Layer 14: 1×1 Conv to num_classes
    x = layers.Conv2D(num_classes, 1, padding='same')(x)

    # Layer 15: Bilinear interpolation ×8
    def bilinear_upsample(x):
        input_size = tf.shape(x)[1:3]
        output_size = input_size * 8
        return tf.image.resize(x, output_size, method='bilinear')

    outputs = layers.Lambda(bilinear_upsample)(x)

    return Model(inputs, outputs)

# Create model for 1-channel input
CFPBlock = build_model_1channel(input_shape=(256, 256, 1), num_classes=1)

# Print model summary
CFPBlock.summary()

# Calculate total parameters
total_params = CFPBlock.count_params()
print(f"\nTotal parameters: {total_params:,} ({total_params/1e6:.3f}M)")

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_234 (Conv2D) │ (None, 128, 128,  │        216 │ input_layer_8[0]… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         96 │ conv2d_234[0][0]  │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_111 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_235 (Conv2D) │ (None, 128, 128,  │      5,184 │ re_lu_111[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         96 │ conv2d_235[0][0]  │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_112 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_236 (Conv2D) │ (None, 128, 128,  │      5,184 │ re_lu_112[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         96 │ conv2d_236[0][0]  │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_113 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_237 (Conv2D) │ (None, 64, 64,    │     10,368 │ re_lu_113[0][0]   │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        192 │ conv2d_237[0][0]  │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_114 (ReLU)    │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1, 1, 48)  │          0 │ re_lu_114[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_239 (Conv2D) │ (None, 1, 1, 24)  │      1,152 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1, 1, 24)  │         96 │ conv2d_239[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_115 (ReLU)    │ (None, 1, 1, 24)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 629,945 (2.40 MB)

 Trainable params: 626,553 (2.39 MB)

 Non-trainable params: 3,392 (13.25 KB)


Total parameters: 629,945 (0.630M)


In [65]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
CFPBlock.load_weights('/content/1_Weights on DeepCrack dataset/CFPBlock_on_HRCDS_2025.11.10.h5')


In [66]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [67]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [68]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = CFPBlock.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_CFPBlock"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 841ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [69]:
## Download predictions and trained model
!zip -r /content/predection_of_CFPBlock.zip /content/predection_of_CFPBlock
from google.colab import files
files.download("/content/predection_of_CFPBlock.zip")


  adding: content/predection_of_CFPBlock/ (stored 0%)
  adding: content/predection_of_CFPBlock/image_3.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_63.png (deflated 0%)
  adding: content/predection_of_CFPBlock/image_84.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_16.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_2.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_12.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_31.png (deflated 0%)
  adding: content/predection_of_CFPBlock/image_9.png (deflated 0%)
  adding: content/predection_of_CFPBlock/image_21.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_69.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_77.png (deflated 0%)
  adding: content/predection_of_CFPBlock/image_75.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_51.png (deflated 1%)
  adding: content/predection_of_CFPBlock/image_48.png (deflated 0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ULite**

In [70]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [71]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def ULite(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    e1 = _conv_block(inputs, 32)  # Increased filters
    e2 = _conv_block(layers.MaxPooling2D(2)(e1), 32)  # Increased filters
    e3 = _conv_block(layers.MaxPooling2D(2)(e2), 64)  # Increased filters

    # Bridge
    bridge = _conv_block(layers.MaxPooling2D(2)(e3), 128)  # Increased filters

    # Decoder with skip connections
    d3 = layers.Conv2DTranspose(132, 2, strides=2)(bridge)  # Increased filters
    d3 = layers.Concatenate()([d3, e3])
    d3 = _conv_block(d3, 132)  # Increased filters

    d2 = layers.Conv2DTranspose(32, 2, strides=2)(d3)  # Increased filters
    d2 = layers.Concatenate()([d2, e2])
    d2 = _conv_block(d2, 32)  # Increased filters

    d1 = layers.Conv2DTranspose(32, 2, strides=2)(d2)  # Increased filters
    d1 = layers.Concatenate()([d1, e1])
    d1 = _conv_block(d1, 32)  # Increased filters

    # Final convolution - binary output
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    # Model summary
    model = Model(inputs, outputs)
    model.summary()

    return model

def _conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x


# The model
ULite = ULite()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_264 (Conv2D) │ (None, 256, 256,  │        320 │ input_layer_9[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_264[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_132 (ReLU)    │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_265 (Conv2D) │ (None, 256, 256,  │      9,248 │ re_lu_132[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_265[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_133 (ReLU)    │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_32    │ (None, 128, 128,  │          0 │ re_lu_133[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_266 (Conv2D) │ (None, 128, 128,  │      9,248 │ max_pooling2d_32… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_266[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_134 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_267 (Conv2D) │ (None, 128, 128,  │      9,248 │ re_lu_134[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_267[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_135 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_33    │ (None, 64, 64,    │          0 │ re_lu_135[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_268 (Conv2D) │ (None, 64, 64,    │     18,496 │ max_pooling2d_33… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_268[0][0]

 Total params: 842,701 (3.21 MB)

 Trainable params: 840,893 (3.21 MB)

 Non-trainable params: 1,808 (7.06 KB)

In [72]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
ULite.load_weights('/content/1_Weights on DeepCrack dataset/ULite_on_HRCDS_2025.11.5.h5')


In [73]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [74]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [75]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = ULite.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_ULite"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step
✅ All predictions saved (black crack on white background). No images displayed.


In [76]:
## Download predictions and trained model
!zip -r /content/predection_of_ULite.zip /content/predection_of_ULite
from google.colab import files
files.download("/content/predection_of_ULite.zip")


  adding: content/predection_of_ULite/ (stored 0%)
  adding: content/predection_of_ULite/image_3.png (deflated 6%)
  adding: content/predection_of_ULite/image_63.png (deflated 4%)
  adding: content/predection_of_ULite/image_84.png (deflated 5%)
  adding: content/predection_of_ULite/image_16.png (deflated 6%)
  adding: content/predection_of_ULite/image_2.png (deflated 3%)
  adding: content/predection_of_ULite/image_12.png (deflated 6%)
  adding: content/predection_of_ULite/image_31.png (deflated 5%)
  adding: content/predection_of_ULite/image_9.png (deflated 12%)
  adding: content/predection_of_ULite/image_21.png (deflated 5%)
  adding: content/predection_of_ULite/image_69.png (deflated 6%)
  adding: content/predection_of_ULite/image_77.png (deflated 4%)
  adding: content/predection_of_ULite/image_75.png (deflated 4%)
  adding: content/predection_of_ULite/image_51.png (deflated 4%)
  adding: content/predection_of_ULite/image_48.png (deflated 5%)
  adding: content/predection_of_ULite/ima

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **UNetSmall**

In [77]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [78]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def UNetSmall(input_shape=(256, 256, 1)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    e1 = _conv_block_unet(inputs, 16)  # Reduced from 32 to 16
    e2 = _conv_block_unet(layers.MaxPooling2D(2)(e1), 32)  # Reduced from 64 to 32
    e3 = _conv_block_unet(layers.MaxPooling2D(2)(e2), 64)  # Reduced from 128 to 64

    # Bridge
    bridge = _conv_block_unet(layers.MaxPooling2D(2)(e3), 128)  # Reduced from 256 to 128

    # Decoder
    d3 = layers.Conv2DTranspose(64, 2, strides=2)(bridge)  # Reduced from 128 to 64
    d3 = layers.Concatenate()([d3, e3])
    d3 = _conv_block_unet(d3, 64)  # Reduced from 128 to 64

    d2 = layers.Conv2DTranspose(32, 2, strides=2)(d3)  # Reduced from 64 to 32
    d2 = layers.Concatenate()([d2, e2])
    d2 = _conv_block_unet(d2, 32)  # Reduced from 64 to 32

    d1 = layers.Conv2DTranspose(16, 2, strides=2)(d2)  # Reduced from 32 to 16
    d1 = layers.Concatenate()([d1, e1])
    d1 = _conv_block_unet(d1, 16)  # Reduced from 32 to 16

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    model = Model(inputs, outputs)
    model.summary()

    return model

def _conv_block_unet(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

# The model
UNetSmall = UNetSmall()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_279 (Conv2D) │ (None, 256, 256,  │        160 │ input_layer_10[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_279[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_146 (ReLU)    │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_280 (Conv2D) │ (None, 256, 256,  │      2,320 │ re_lu_146[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ conv2d_280[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_147 (ReLU)    │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_35    │ (None, 128, 128,  │          0 │ re_lu_147[0][0]   │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_281 (Conv2D) │ (None, 128, 128,  │      4,640 │ max_pooling2d_35… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_281[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_148 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_282 (Conv2D) │ (None, 128, 128,  │      9,248 │ re_lu_148[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_282[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_149 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_36    │ (None, 64, 64,    │          0 │ re_lu_149[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_283 (Conv2D) │ (None, 64, 64,    │     18,496 │ max_pooling2d_36… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_283[0][0]

 Total params: 484,561 (1.85 MB)

 Trainable params: 483,153 (1.84 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [79]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
UNetSmall.load_weights('/content/1_Weights on DeepCrack dataset/UNetSmall_on_HRCDS_2025.11.5.h5')


In [80]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [81]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [82]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = UNetSmall.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_UNetSmall"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 428ms/step
✅ All predictions saved (black crack on white background). No images displayed.


In [83]:
## Download predictions and trained model
!zip -r /content/predection_of_UNetSmall.zip /content/predection_of_UNetSmall
from google.colab import files
files.download("/content/predection_of_UNetSmall.zip")


  adding: content/predection_of_UNetSmall/ (stored 0%)
  adding: content/predection_of_UNetSmall/image_3.png (deflated 4%)
  adding: content/predection_of_UNetSmall/image_63.png (deflated 5%)
  adding: content/predection_of_UNetSmall/image_84.png (deflated 6%)
  adding: content/predection_of_UNetSmall/image_16.png (deflated 5%)
  adding: content/predection_of_UNetSmall/image_2.png (deflated 4%)
  adding: content/predection_of_UNetSmall/image_12.png (deflated 7%)
  adding: content/predection_of_UNetSmall/image_31.png (deflated 5%)
  adding: content/predection_of_UNetSmall/image_9.png (deflated 10%)
  adding: content/predection_of_UNetSmall/image_21.png (deflated 4%)
  adding: content/predection_of_UNetSmall/image_69.png (deflated 6%)
  adding: content/predection_of_UNetSmall/image_77.png (deflated 4%)
  adding: content/predection_of_UNetSmall/image_75.png (deflated 4%)
  adding: content/predection_of_UNetSmall/image_51.png (deflated 5%)
  adding: content/predection_of_UNetSmall/image_48

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **LiteFusionNet**

In [84]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate

In [85]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def residual_block(x, out_channels, stride=1, name=None):
    """Reduced Residual Block"""
    in_channels = x.shape[-1]

    # Main path
    y = layers.Conv2D(out_channels, kernel_size=3, strides=stride,
                     padding='same', use_bias=False, name=f'{name}_conv1')(x)
    y = layers.BatchNormalization(name=f'{name}_bn1')(y)
    y = layers.ReLU(name=f'{name}_relu1')(y)

    y = layers.Conv2D(out_channels, kernel_size=3, strides=1,
                     padding='same', use_bias=False, name=f'{name}_conv2')(y)
    y = layers.BatchNormalization(name=f'{name}_bn2')(y)

    # Shortcut path
    if stride != 1 or in_channels != out_channels:
        shortcut = layers.Conv2D(out_channels, kernel_size=1, strides=stride,
                                use_bias=False, name=f'{name}_shortcut_conv')(x)
        shortcut = layers.BatchNormalization(name=f'{name}_shortcut_bn')(shortcut)
    else:
        shortcut = x

    # Add and activate
    y = layers.Add(name=f'{name}_add')([y, shortcut])
    y = layers.ReLU(name=f'{name}_relu2')(y)

    return y

def dga_block(x, name=None):
    """Simplified Dual-Gated Attention Block"""
    channels = x.shape[-1]

    # Simplified channel attention
    channel_att = layers.GlobalAveragePooling2D(keepdims=True, name=f'{name}_gap')(x)
    channel_att = layers.Conv2D(max(8, channels // 16), kernel_size=1, activation='relu',
                               name=f'{name}_channel_conv1')(channel_att)
    channel_att = layers.Conv2D(channels, kernel_size=1, activation='sigmoid',
                               name=f'{name}_channel_conv2')(channel_att)

    # Simplified spatial attention
    spatial_att = layers.Conv2D(1, kernel_size=1, activation='sigmoid',
                               name=f'{name}_spatial_conv')(x)

    # Apply attention
    x = layers.Multiply(name=f'{name}_channel_att')([x, channel_att])
    x = layers.Multiply(name=f'{name}_spatial_att')([x, spatial_att])

    return x

def dvm_block(x, name=None):
    """Simplified Dual-View Module Block"""
    channels = x.shape[-1]

    # Simplified depthwise separable convolution
    x = layers.DepthwiseConv2D(kernel_size=3, padding='same', name=f'{name}_depthwise')(x)
    x = layers.Conv2D(channels, kernel_size=1, name=f'{name}_pointwise')(x)
    x = layers.BatchNormalization(name=f'{name}_bn')(x)
    x = layers.ReLU(name=f'{name}_relu')(x)

    return x

def easpp_block(x, out_channels=64, name=None):
    """Reduced Enhanced Atrous Spatial Pyramid Pooling Block"""
    # Reduced number of branches
    conv1 = layers.Conv2D(out_channels, kernel_size=1, use_bias=False, name=f'{name}_conv1')(x)
    conv1 = layers.BatchNormalization(name=f'{name}_bn1')(conv1)
    conv1 = layers.ReLU(name=f'{name}_relu1')(conv1)

    conv2 = layers.Conv2D(out_channels, kernel_size=3, padding='same', dilation_rate=6,
                         use_bias=False, name=f'{name}_conv2')(x)
    conv2 = layers.BatchNormalization(name=f'{name}_bn2')(conv2)
    conv2 = layers.ReLU(name=f'{name}_relu2')(conv2)

    # Global average pooling branch
    gap = layers.GlobalAveragePooling2D(keepdims=True, name=f'{name}_gap')(x)
    gap = layers.Conv2D(out_channels, kernel_size=1, use_bias=False, name=f'{name}_gap_conv')(gap)
    gap = layers.BatchNormalization(name=f'{name}_gap_bn')(gap)
    gap = layers.ReLU(name=f'{name}_gap_relu')(gap)
    gap = layers.UpSampling2D(size=(x.shape[1], x.shape[2]), interpolation='bilinear', name=f'{name}_upsample')(gap)

    # Concatenate and fuse
    concatenated = layers.Concatenate(axis=-1, name=f'{name}_concat')([conv1, conv2, gap])
    output = layers.Conv2D(out_channels, kernel_size=1, use_bias=False, name=f'{name}_fuse_conv')(concatenated)
    output = layers.BatchNormalization(name=f'{name}_fuse_bn')(output)
    output = layers.ReLU(name=f'{name}_fuse_relu')(output)

    return output

def decoder_block(x, skip, out_channels, name=None):
    """Decoder Block with skip connection"""
    # Upsample
    x = layers.Conv2DTranspose(out_channels, kernel_size=2, strides=2, name=f'{name}_transpose')(x)

    # Concatenate with skip connection
    x = layers.Concatenate(axis=-1, name=f'{name}_concat')([x, skip])

    # Residual block
    x = residual_block(x, out_channels, stride=1, name=f'{name}_residual')

    return x

def LiteFusionNet(input_shape=(256, 256, 1)):
    """Ultra-Light LiteFusionNet Model"""
    # Input
    inputs = layers.Input(shape=input_shape, name='input')

    # Initial convolution (reduced channels)
    x = layers.Conv2D(16, kernel_size=3, strides=2, padding='same', use_bias=False, name='incoder1_conv')(inputs)
    x = layers.BatchNormalization(name='incoder1_bn')(x)
    x = layers.ReLU(name='incoder1_relu')(x)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding='same', name='incoder1_pool')(x)
    skip1 = x  # /4

    # Encoder blocks with reduced channels
    x = residual_block(x, 16, stride=2, name='incoder2')  # /8
    skip2 = x

    x = residual_block(x, 32, stride=2, name='incoder3')  # /16
    skip3 = x

    x = residual_block(x, 64, stride=2, name='incoder4')  # /32
    skip4 = x

    x = residual_block(x, 64, stride=2, name='incoder5')  # /64
    x = dga_block(x, name='incoder5_dga')
    skip5 = x

    x = residual_block(x, 128, stride=2, name='incoder6')  # /128
    x = dga_block(x, name='incoder6_dga')

    # Bridge with reduced EASPP
    x = easpp_block(x, out_channels=64, name='easpp')

    # Decoder blocks with reduced channels
    x = decoder_block(x, skip5, 32, name='decoder5')  # /64
    x = decoder_block(x, skip4, 32, name='decoder4')  # /32
    x = decoder_block(x, skip3, 16, name='decoder3')   # /16
    x = decoder_block(x, skip2, 16, name='decoder2')   # /8

    # Final decoder
    x = layers.Conv2DTranspose(16, kernel_size=2, strides=2, name='decoder1_transpose')(x)
    x = residual_block(x, 8, stride=1, name='decoder1_residual')  # /4

    # Final convolution - output with same number of channels as input
    x = layers.Conv2D(input_shape[-1], kernel_size=1, activation='sigmoid', name='output')(x)
    x = layers.UpSampling2D(size=4, interpolation='bilinear', name='final_upsample')(x)

    # Create model
    model = Model(inputs=inputs, outputs=x, name='LiteFusionNet')

    return model

# Create and test the model
if __name__ == "__main__":
    # Create the model
    LiteFusionNet = LiteFusionNet(input_shape=(256, 256, 1))

    # Display model summary
    LiteFusionNet.summary()

    # Calculate total parameters
    total_params = LiteFusionNet.count_params()
    print(f"\nTotal parameters: {total_params:,} ({total_params/1e6:.3f}M)")


Model: "LiteFusionNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 256, 256,  │          0 │ -                 │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder1_conv       │ (None, 128, 128,  │        144 │ input[0][0]       │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder1_bn         │ (None, 128, 128,  │         64 │ incoder1_conv[0]… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder1_relu       │ (None, 128, 128,  │          0 │ incoder1_bn[0][0] │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder1_pool       │ (None, 64, 64,    │          0 │ incoder1_relu[0]… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_conv1      │ (None, 32, 32,    │      2,304 │ incoder1_pool[0]… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_bn1        │ (None, 32, 32,    │         64 │ incoder2_conv1[0… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_relu1      │ (None, 32, 32,    │          0 │ incoder2_bn1[0][… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_conv2      │ (None, 32, 32,    │      2,304 │ incoder2_relu1[0… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_shortcut_… │ (None, 32, 32,    │        256 │ incoder1_pool[0]… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_bn2        │ (None, 32, 32,    │         64 │ incoder2_conv2[0… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_shortcut_… │ (None, 32, 32,    │         64 │ incoder2_shortcu… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_add (Add)  │ (None, 32, 32,    │          0 │ incoder2_bn2[0][… │
│                     │ 16)               │            │ incoder2_shortcu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder2_relu2      │ (None, 32, 32,    │          0 │ incoder2_add[0][… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder3_conv1      │ (None, 16, 16,    │      4,608 │ incoder2_relu2[0… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder3_bn1        │ (None, 16, 16,    │        128 │ incoder3_conv1[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ incoder3_relu1      │ (None, 16, 16,    │          0 │ incoder3_bn1[0][

 Total params: 611,387 (2.33 MB)

 Trainable params: 608,395 (2.32 MB)

 Non-trainable params: 2,992 (11.69 KB)


Total parameters: 611,387 (0.611M)


In [86]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the model weights
LiteFusionNet.load_weights('/content/1_Weights on DeepCrack dataset/LiteFusionNet_on_HRCDS_2025.11.5.h5')


In [87]:
test_files3 = []
for i in range(1,101):
  test_files3.append("/content/HRCDS/test_image/"+str(i)+".jpg")
test_files3

['/content/HRCDS/test_image/1.jpg',
 '/content/HRCDS/test_image/2.jpg',
 '/content/HRCDS/test_image/3.jpg',
 '/content/HRCDS/test_image/4.jpg',
 '/content/HRCDS/test_image/5.jpg',
 '/content/HRCDS/test_image/6.jpg',
 '/content/HRCDS/test_image/7.jpg',
 '/content/HRCDS/test_image/8.jpg',
 '/content/HRCDS/test_image/9.jpg',
 '/content/HRCDS/test_image/10.jpg',
 '/content/HRCDS/test_image/11.jpg',
 '/content/HRCDS/test_image/12.jpg',
 '/content/HRCDS/test_image/13.jpg',
 '/content/HRCDS/test_image/14.jpg',
 '/content/HRCDS/test_image/15.jpg',
 '/content/HRCDS/test_image/16.jpg',
 '/content/HRCDS/test_image/17.jpg',
 '/content/HRCDS/test_image/18.jpg',
 '/content/HRCDS/test_image/19.jpg',
 '/content/HRCDS/test_image/20.jpg',
 '/content/HRCDS/test_image/21.jpg',
 '/content/HRCDS/test_image/22.jpg',
 '/content/HRCDS/test_image/23.jpg',
 '/content/HRCDS/test_image/24.jpg',
 '/content/HRCDS/test_image/25.jpg',
 '/content/HRCDS/test_image/26.jpg',
 '/content/HRCDS/test_image/27.jpg',
 '/content

In [88]:
X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)

#Grayscale test
GRAY=True
if GRAY:
  X_test3= np.zeros((len(test_files3),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.uint8)

In [89]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import re  # ✅ For robust filename sorting

# ================================
# Parameters
# ================================
GRAY = True                     # True if input images are grayscale
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1                # 1 for grayscale, 3 for RGB
# ================================

# Folder with test images
folder_path = '/content/HRCDS/test_image'

# ================================
# Load test image files
# ================================
def extract_number(filename):
    """Extract first number found in filename for sorting."""
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else 0

test_files = sorted(os.listdir(folder_path), key=extract_number)

X_test = []

for file in test_files:
    img_path = os.path.join(folder_path, file)

    if GRAY:
        # Read as grayscale
        img = cv2.imread(img_path, 0)
        # Resize and reshape
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
    else:
        # Read as RGB
        img = imread(img_path)[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    X_test.append(img)

X_test = np.array(X_test)

# ================================
# Make predictions using your model
# ================================
# Assuming `model_SegNet` is already defined and loaded
predictions = LiteFusionNet.predict(X_test)


# Save results (no display)
# ================================
output_dir = "/content/predection_of_LiteFusionNet"
os.makedirs(output_dir, exist_ok=True)

for i, prediction in enumerate(predictions):
    pred_display = prediction.squeeze()         # ✅ Correct shape (256, 256)
    inverted_prediction = 1.0 - pred_display    # ✅ Crack = black, BG = white

    # Save directly without showing
    output_path = os.path.join(output_dir, f"image_{i+1}.png")
    plt.imsave(output_path, inverted_prediction, cmap='gray')

print("✅ All predictions saved (black crack on white background). No images displayed.")

4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step
✅ All predictions saved (black crack on white background). No images displayed.


In [90]:
## Download predictions and trained model
!zip -r /content/predection_of_LiteFusionNet.zip /content/predection_of_LiteFusionNet
from google.colab import files
files.download("/content/predection_of_LiteFusionNet.zip")


  adding: content/predection_of_LiteFusionNet/ (stored 0%)
  adding: content/predection_of_LiteFusionNet/image_3.png (deflated 4%)
  adding: content/predection_of_LiteFusionNet/image_63.png (deflated 5%)
  adding: content/predection_of_LiteFusionNet/image_84.png (deflated 4%)
  adding: content/predection_of_LiteFusionNet/image_16.png (deflated 4%)
  adding: content/predection_of_LiteFusionNet/image_2.png (deflated 2%)
  adding: content/predection_of_LiteFusionNet/image_12.png (deflated 8%)
  adding: content/predection_of_LiteFusionNet/image_31.png (deflated 4%)
  adding: content/predection_of_LiteFusionNet/image_9.png (deflated 13%)
  adding: content/predection_of_LiteFusionNet/image_21.png (deflated 3%)
  adding: content/predection_of_LiteFusionNet/image_69.png (deflated 7%)
  adding: content/predection_of_LiteFusionNet/image_77.png (deflated 5%)
  adding: content/predection_of_LiteFusionNet/image_75.png (deflated 3%)
  adding: content/predection_of_LiteFusionNet/image_51.png (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>